# P300 speller (calibration mode)

- Target letter is known each time, so every falsh can be labelled as target or non-target

## Packages

- source .speller/bin/activate
- deactivate

- python 3.10
- psychopy, pylsl

## Speller parameters

In [2]:
from psychopy import visual, core, event
import random
import time

GRID_ROWS = 6
GRID_COLS = 6

FLASH_DUR  = 0.100 # duration of each flash in seconds
ISI = 0.075 # time between end of one flash and start of next) in seconds
N_SEQS = 10 # number of sequences (i.e. how many times to flash each row/column) 
TEXT_H = 36 # letter size

KeyboardInterrupt: 

## Build window and stimuli

In [ ]:
win = visual.Window([100, 700]', units='pix', fullscr=True, color='[-0.05]*3)

#position for each cell
grid_w, grid_h = 700, 500
x0, y0 = -grid_w/2, grid_h/2
dx = grid_w/(GRID_COLS-1)
dy = grid_h/(GRID_ROWS-1)

# create text stimuli for each symbol
cells = []
for r in range(GRID_ROWS):
    for c in range(GRID_COLS):
        idx = r*GRID_COLS + c
        x = x0 + c*dx
        y = y0 - r*dy
        stim = visual.TextStim(win, text=SYMS(idx), pos=(x,y), height=TEXT_H, color='white', bold=True)
        cells.append(stim)

# instruction + status line
instr = visual.TextStim(win, text='Focus on the target symbol and count how many times it flashes. Press SPACE to start and ESC to quit', pos=(0, -3000), height=24, color='white')
status = visual.TextStim(win, text='', pos=(0, -300), height=22, color='white') 

# highlight: None, ("row, r") or ("col",c)
def draw_grid(highlight=None):
    for i, stim in enumerate(cells):
        # apply highlight
        if highlight is not None:
            kind, k = highlight
            if kind == 'row':
                for c in range(GRID_COLS):
                    cells[k*GRID_COLS + c].color = 'yellow'
            elif kind == 'col':
                for r in range(GRID_ROWS):
                    cells[r*GRID_COLS + k].color = 'yellow'
        for stim in cells:
            stim.draw()
        status.draw()

## Event logging (local)

In [ ]:
event_log = [] # list of dicts

def log_event(ev_type, value):
    event_log.append({'t': time.time(), 'type': ev_type, 'value': value})

## Flash routines

In [ ]:
# kind: "row" or "col", idx: row/col index
def flash(kind, idx, label=''):
    # highlight ON
    status.text = label
    draw_grid(highlight=(kind, idx))
    win.flip()
    log_event('flash_on', f'{kind}_{idx}')

    core.wait(FLASH_DUR)

    # highlight OFF
    draw_grid(highlight=None)
    win.flip()
    log_event('flash_off', f'{kind}_{idx}')

    core.wait(ISI)

# runs N_SEQS sequences for one target character. Target is defined by its row?col in the grid
def run_trial(target_char):
    if target_char not in SYMS:
        raise ValueError(f'Invalid target character: {target_char}')
    
    target_idx = SYMS.index(target_char)
    tr = target_idx // GRID_COLS
    tc = target_idx % GRID_COLS

    status.text = f'Focus on: {target_char}'
    draw_grid(None)
    win.flip()
    core.wait(1.0)
    log_event('target', target_char)

    # each sequence is 6 rows and 6 cols in random order (12 flashes)
    for s in range(N_SEQS):
        order = [("row", r) for r in range(GRID_ROWS)] + [("col", c) for c in range(GRID_COLS)]
        random.shuffle(order)

        for kind, idx in order:
            flash(kind, idx)
        
        # abort option
        if 'escape' in event.getKeys():
            log_event('abort', '')
            return False
    # after sequence, short break
    status.text = 'Break...'
    draw_grid(None)
    win.flip()
    core.wait(0.8)
    return True

## Main

In [ ]:
instr.draw()
draw_grid(None)
win.flip()

# wait to start
while True:
    keys = event.getKeys()
    if 'space' in keys:
        break
    if 'escape' in keys:
        win.close()
        core.quit()

# example run with target 'PLAY'
word = 'PLAY'
for ch in word:
    ok = run_trial(ch)
    if not ok:
        break

# end screen
status.text = 'Thanks for participating! Press ESC to quit.'
draw_grid(None)
win.flip()
while 'escape' not in event.getKeys():
    core.wait(0.05)

win.close()

# print log summary
print(f'Logged {len(event_log)} events:')
print(event_log[:5])